In [26]:
from wordcloud import WordCloud
import sys
import pandas as pd
from src import DATA_DIR
import numpy as np
from PIL import Image
from scipy.ndimage import gaussian_gradient_magnitude
import os

In [27]:
def generate_by_dict(data_path, mask_path, output_name):
    df = pd.read_csv(data_path)
    word_dict = {df.iloc[i, 0]:df.iloc[i, 1] for i in range(len(df))}
    
    parrot_color = np.array(Image.open(mask_path))
    parrot_color = parrot_color[::3, ::3]

    parrot_mask = parrot_color.copy()
    parrot_mask[parrot_mask.sum(axis=2) == 0] = 255

    edges = np.mean([gaussian_gradient_magnitude(parrot_color[:, :, i] / 255., 2) for i in range(3)], axis=0)
    parrot_mask[edges > .08] = 255
    
    
    # Generate_word_cloud_from_text
    if not mask_path:
        wc = WordCloud(
            width=670,
            height=670,
            background_color='rgba(0,0,0,0)',
                font_path=str(DATA_DIR / 'data/font/Estedad-Regular.ttf'),
            max_font_size=150,
            max_words=1000,
            regexp=r"\w[\w'\\s]+",
        ).generate_from_frequencies(word_dict)
        
    else:
        wc = WordCloud(
            mask=parrot_mask,
            width=670,
            height=670,
            background_color='rgba(0,0,0,0)',
            font_path=str(DATA_DIR / 'data/font/Estedad-Regular.ttf'),
            max_font_size=150,
            max_words=1000,
            regexp=r"\w[\w'\\s]+",
        ).generate_from_frequencies(word_dict)


    with open(output_name, 'w') as f:
        f.write(wc.to_svg())

In [33]:
generate_by_dict(
    DATA_DIR / 'data/data2.csv',
    DATA_DIR / 'data/mask/02.png',
    DATA_DIR / 'output/pic2.svg'
)